## **CS333 Technical Demo**

MCMC algorithm primarily authored by Noah Harris. Formatting added by Luke Nam.

### **Importing Dependencies**

Prior to running this notebook, make sure that all the dependent libraries have been installed on your machine. We'll be using `networkx` for undirected graphs, `numpy` for matrix operations, and `matplotlib` for graph visualizations.

In [3]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import copy
import plotly.graph_objects as go
from IPython.display import display, HTML
seed_value=46

### **Importing the Graph**

Let's generate the graph from the helper function in generate_data.py

In [4]:
from generate_data import *
graph = get_colorado_graph()

In [18]:
n = len(graph.nodes)
graph_nodes = graph.nodes

In [16]:
def divide_into_subgraphs(graph, n):
    # Ensure n is a positive integer less than or equal to the number of nodes in the graph
    if not isinstance(n, int) or n <= 0 or n > len(graph.nodes):
        raise ValueError("Invalid value for n. It should be a positive integer less than or equal to the number of nodes in the graph.")

    # Get the list of nodes in the graph
    all_nodes = list(graph.nodes)

    # Calculate the number of nodes in each subgraph
    nodes_per_subgraph = len(all_nodes) // n

    # Initialize empty subgraph list
    subgraphs = []

    # possible colors
    colors = ["orange", "green", "red", "blue", "yellow", "pink"]

    # Divide nodes into n subgraphs, coloring nodes accordingly
    for i in range(n):
        start_index = i * nodes_per_subgraph
        end_index = (i + 1) * nodes_per_subgraph if i < n - 1 else len(all_nodes)
        subgraph_nodes = all_nodes[start_index:end_index]
        for node in subgraph_nodes:
            graph.nodes[node]["color"] = colors[i]

        subgraph = graph.subgraph(subgraph_nodes)
        subgraphs.append(subgraph)

    return subgraphs

In [17]:
def visualize_subgraphs(graph, subgraphs, highlighted_edge="none"):
    pos = nx.spring_layout(graph, seed=seed_value)  # You can use a different layout if you prefer

    # Draw the original graph
    nx.draw(graph, pos, with_labels=True, font_weight='bold', node_color='lightgray', edge_color='gray', node_size=700)

    # Draw each subgraph with a different color
    for i, subgraph in enumerate(subgraphs, start=1):
        node_ids = list(subgraph.nodes.keys())
        color = subgraph.nodes[node_ids[0]]["color"]
        nx.draw(subgraph, pos, with_labels=True, font_weight='bold', node_color = color, 
                edge_color=f'C{i}', node_size=700)

    # Highlight the specified edge if it's not "none"
    if highlighted_edge != "none":
        nx.draw_networkx_edges(graph, pos, edgelist=[highlighted_edge], edge_color='purple', width=2)

    plt.show()

In [280]:
def get_edges_between_subgraphs(graph, subgraphs):
    edges_between_subgraphs = []
    for i in range(len(subgraphs)):
        for j in range(i + 1, len(subgraphs)):
            edges_between_subgraphs.extend(nx.edge_boundary(graph, subgraphs[i], subgraphs[j]))
    return edges_between_subgraphs

In [281]:
def get_random_edge_between_subgraphs(graph, subgraphs):
    edges_between_subgraphs = get_edges_between_subgraphs(graph, subgraphs)
    if edges_between_subgraphs:
        return random.choice(edges_between_subgraphs)
    else:
        return None

In [26]:
def create_proposed_state(graph, subgraphs, conflicted_edge):
    # Create deep copies of the graph and subgraphs
    modified_graph = copy.deepcopy(graph)
    modified_subgraphs = [subgraph.copy() for subgraph in subgraphs]

    # Get the nodes connected by the conflicted edge
    node1, node2 = conflicted_edge

    # Choose one of the nodes randomly
    node_to_move = random.choice([node1, node2])

    # Find the index of the subgraph containing the node to move
    index_with_node = next((i for i, subgraph in enumerate(modified_subgraphs) if node_to_move in subgraph), None)

    # Find the node not selected in the random choice
    node_not_selected = node2 if node_to_move == node1 else node1

    if index_with_node is not None:
        # Remove the node from its current subgraph
        modified_subgraphs[index_with_node].remove_node(node_to_move)

        # Find the index of the subgraph not containing the node
        index_other_node = next((i for i, subgraph in enumerate(modified_subgraphs) if node_not_selected in subgraph), None)

        # Add the node to the other subgraph
        mod_sub = modified_subgraphs[index_other_node]
        new_color = mod_sub.nodes[list(mod_sub.nodes())[0]]["color"]
        mod_sub.add_node(node_to_move)
        mod_sub.nodes[node1]["color"] = new_color
        

    return modified_graph, modified_subgraphs


In [22]:
def calculate_total_population(graph):
    total_population = sum(graph.nodes[node]['population'] for node in graph.nodes)
    return total_population

In [23]:
c_pop=.3
def populationscore(graph,subgraphs):
    totalsum=0
    districtaverage=total_population/len(subgraphs)
    for i, subgraph in enumerate(subgraphs, start=1):
        # Calculate the sum of populations for each subgraph
        population_sum = sum(graph.nodes[node]['population'] for node in subgraph.nodes)
        totalsum+=(population_sum-districtaverage)**2
    return(c_pop*totalsum)

In [24]:
def PVIscore(graph,subgraphs):
    return 0

In [294]:
lambda_J=1
def totalscorefunction(graph,subgraph):
    return lambda_J*populationscore(graph,subgraph)+(1-lambda_J)*PVIscore(graph,subgraph)

In [295]:
beta=0.002
def one_iteration_of_MCMC(graph,subgraphs):
    con1=len(get_edges_between_subgraphs(graph,subgraphs))
    conflicted_edge = get_random_edge_between_subgraphs(graph, subgraphs)
    proposed_graph, proposed_subgraphs = create_proposed_state(test_graph, testsubgraphs, conflicted_edge)
    con2=len(get_edges_between_subgraphs(proposed_graph,proposed_subgraphs))
    probfromscorefunctions=np.exp(-1*beta*(totalscorefunction(proposed_graph,proposed_subgraphs)-totalscorefunction(graph,subgraphs))) 
    transitionprobability=min(1,con1/con2,float(probfromscorefunctions))
    print(transitionprobability)
    # Generate a random number between 0 and 1
    random_number = np.random.rand()

    # Check if the random number is less than the transition probability
    if random_number < transitionprobability:
        return proposed_graph, proposed_subgraphs
    else:
        return graph, subgraphs

In [293]:
def Redistricting_MCMC(graph, subgraphs,num_iterations):
    # Perform MCMC iterations
    for _ in range(num_iterations):
        # Call one_iteration_of_MCMC for each iteration
        graph, subgraphs = one_iteration_of_MCMC(graph, subgraphs)
    return graph, subgraphs